In [5]:
import random
import json
import os
from core.document_retrieval import VectorDB, Retriever
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
source = 'ars_electronica_prizewinners_ru.json'
path = 'not_posted.txt'

vector_db = VectorDB(source)
vector_db.load()
vector_db.create_db()
retriever = Retriever(vector_db)

with open(source, 'r', encoding='utf-8') as f:
    data = json.load(f)

with open(path, 'r') as f:
    not_posted = f.readline().split(',')

key = random.choice(not_posted)
print(f'Key is {key}')
artwork = data[key]

print("Source:\n")
print(artwork["description_ru"], "\n")

res = retriever.get_top_k(artwork['description_ru'], key, 2)

print("Recs:\n")
for example in res:
    print(example.page_content, "\n")
    
    
def delete_apostrophe(text):
    return text.replace("'", "")

art_name_main = artwork['name']
authors_main = artwork['authors']
year_main = artwork['year']
description_main = delete_apostrophe(artwork['description'])
image_main = artwork['img_list'][0]

art_name_0 = res[0].metadata['name']
authors_0 = res[0].metadata['authors']
year_0 = res[0].metadata['year']
description_0 = delete_apostrophe(res[0].metadata['description'])
image_0 = res[0].metadata['img_list'][0]

art_name_1 = res[1].metadata['name']
authors_1 = res[1].metadata['authors']
year_1 = res[1].metadata['year']
description_1 = delete_apostrophe(res[1].metadata['description'])
image_1 = res[1].metadata['img_list'][0]

Key is 23722
Source:

С 1975 года я создаю движущиеся изображения с помощью новых технологий, особенно систем трехмерной компьютерной анимации. Моя работа сосредоточена на изучении естественных движений и роли человеческих жестов и выражений в языке общения. Я рассматриваю развитие новых технологий как расширение человеческого разума и тела и комментирую эту взаимосвязь. «Установившееся состояние» определяется как стабильное состояние, которое не меняется с течением времени или при котором изменение в одном направлении постоянно уравновешивается изменить в другом. Тонкие паттерны синхронных движений тела возникают, когда люди взаимодействуют друг с другом. Эти телесные ритмы, называемые ритмами взаимодействия, необходимы в каждом человеческом взаимодействии. Когда люди двигают своим телом и вытягивают конечности, они вычерчивают невидимые формы, окружающие их тела, определяя пространство, которое они занимают. Когда два человека взаимодействуют, эти пространственные паттерны также пере

In [30]:
prompt = f'Imagine you are an experienced art critic' \
         f'You are presented with descriptions of three artworks.' \
         f'The first one is a main artwork and two related artworks are similar to the first one.' \
         f'On the first step you should provide a separate review of all the artworks.' \
         f'Artworks also have related images, analyze their content and use this analysis in your reviews.' \
         f'On the second step summarize all of these artworks in terms of similarities and differences.' \
         f'Main artwork: name: {art_name_main}, authors: {authors_main}, year: {year_main}, description: {description_main}' \
         f'Related artwork 1: name: {art_name_0}, authors: {authors_0}, year: {year_0}, description: {description_0}' \
         f'Related artwork 2: name: {art_name_1}, authors: {authors_1}, year: {year_1}, description: {description_1}' \
         f'Give your answer in Russian, but do not translate the names of artists and artworks.' \
         f'Do not divide description and reasoning parts, combine them into the one paragraph' \
         f'Use Markdown to structure your answer. Make the names of artists and artworks bold.' \
         f'Give your answer step-by-step for each artwork.'

response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url":{
                       "url": image_main,
                       "detail": "low"
                    },
                },
                                {
                    "type": "image_url",
                    "image_url":{
                       "url": image_0,
                       "detail": "low"
                    },
                },
                                {
                    "type": "image_url",
                    "image_url":{
                       "url": image_1,
                       "detail": "low"
                    },
                },
            ],
        }
    ],
    max_tokens=1000,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='### Шаг 1: Отдельный обзор каждого произведения\n\n**Работа: Steady State**\n**Автор:** Rebecca Allen\n\n![Image](image1.jpg)\n\n#### Обзор:\n"Steady State" представляет собой сложное мультимедийное произведение, в котором используется 3D-анимация для исследования роли человеческого движения и жестов в общении. Арт-проект выделяется своим вниманием к невидимым формам, создаваемым движением тела, а также взаимодействием этих форм при межличностном общении. Изображение демонстрирует абстрактную фигуру, которая, возможно, изучает или воспроизводит сложные геометрические паттерны движения человеческого тела. Такое визуальное представление подчеркивает технологичный подход и концептуальную глубину работы, где человеческое тело становится источником как эмоционального, так и интеллектуального размышления о будущем взаимодействия человека и технологии.\n\n**Работа: Аdelbrecht**\n**Автор:** Martin Spa